In [36]:
#To install all packages
#!pip3 install pandas seaborn datetime pymongo
#!pip3 install gensim
#!pip3 install python-Levenshtein

In [37]:
import pandas as pd
import seaborn as sns
import gensim

import warnings
warnings.filterwarnings('ignore')


sns.set_style("dark")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [38]:
from datetime import datetime, timedelta   
from pymongo import MongoClient

# uri (uniform resource identifier) defines the connection parameters 
uri = 'url'
# start client to connect to MongoDB server 
client = MongoClient( uri )
db = client.appstore_reviews

In [39]:
pipeline = [{"$match": {"dateISO": {"$gte": datetime(2010, 6, 1, 0, 0)} }},{ "$project": { "loja": 1, "text" : "$comment.text","score" : "$comment.score", "week": { "$week": "$dateISO" }, "dateISO":1, "_id": 0 } }, { "$sort" : { "dateISO" : -1 } }, { "$limit": 5000000}]

mongoList = list(db.reviews.aggregate(pipeline))

In [40]:
data = pd.DataFrame(mongoList)
# data['ts'] = pd.to_datetime(data['ts'], unit='s')
# data.ts = data.ts - timedelta(hours = 3)
data

,loja,dateISO,text,score,week
0,google,2022-12-08 08:26:02,Não consigo cadastrar a senha pois aparece uma...,1,49
1,google,2022-12-08 08:17:09,Muito seguro e fácil,5,49
2,google,2022-12-08 08:14:05,ótimo banco,5,49
3,google,2022-12-08 08:07:35,muito bom,5,49
4,google,2022-12-08 08:07:06,Péssimo essa nova atualização não tem a opção ...,1,49
...,...,...,...,...,...
121673,apple,2017-06-28 20:27:21,SEGUE ABAIXO O COMENTÁRIO DEIXADO AQUI NA QUIN...,5,26
121674,apple,2017-06-04 22:17:22,"Não aceita entrada de gastos manuais, ou seja,...",1,23
121675,apple,2017-05-03 11:30:34,"O app é muito bom, mas não consigo acessar ess...",5,18
121676,apple,2017-03-28 12:39:34,Muito bom esse aplicativo eu não pego mais fil...,5,13


In [41]:
review_text = data.text.apply(gensim.utils.simple_preprocess)

In [42]:
review_text

0         [não, consigo, cadastrar, senha, pois, aparece...
1                                    [muito, seguro, fácil]
2                                            [ótimo, banco]
3                                              [muito, bom]
4         [péssimo, essa, nova, atualização, não, tem, o...
                                ...                        
121673    [segue, abaixo, comentário, deixado, aqui, na,...
121674    [não, aceita, entrada, de, gastos, manuais, ou...
121675    [app, muito, bom, mas, não, consigo, acessar, ...
121676    [muito, bom, esse, aplicativo, eu, não, pego, ...
121677                                               [tpoo]
Name: text, Length: 121678, dtype: object

In [43]:
review_text.loc[0]

['não',
 'consigo',
 'cadastrar',
 'senha',
 'pois',
 'aparece',
 'uma',
 'msg',
 'de',
 'transação',
 'não',
 'efetuada',
 'porque',
 'isso']

In [44]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)


In [45]:
model.build_vocab(review_text, progress_per=1000)

In [46]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)


(4980202, 6722720)

In [47]:
model.save("./word2vec-reviews.model")

In [55]:
model.wv.most_similar("cartao")

[('so', 0.7446611523628235),
 ('nao', 0.6538557410240173),
 ('cartão', 0.6523281335830688),
 ('ja', 0.6179460883140564),
 ('numero', 0.5808336734771729),
 ('mes', 0.5792463421821594),
 ('ai', 0.5673089027404785),
 ('fisico', 0.5653914213180542),
 ('reducir', 0.5623531937599182),
 ('porem', 0.5596981644630432)]

In [54]:
model.wv.similarity(w1="pix", w2="banco")

0.024375597